<a href="https://colab.research.google.com/github/pinballsurgeon/deluxo_adjacency/blob/main/N2_O2_CO2_NVT_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installs

In [12]:
# jax molecular dynamics
!pip install jax-md

# jax based bayesian optimization
#!pip install bayex

#!pip install bayes_opt
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Imports

In [13]:
#                          __         __              /\  .___
#            ____   _____/  |_      |__|____  ___  __)/__| _/
#           / ___\_/ __ \   __\     |  \__  \ \  \/  // __ | 
#          / /_/  >  ___/|  |       |  |/ __ \_>    </ /_/ | 
#          \___  / \___  >__|   /\__|  (____  /__/\_ \____ | 
#          /_____/      \/       \______|    \/      \/    \/ 


from jax.config import config ; config.update('jax_enable_x64', True)
import jax.numpy as np
from jax import random, jit, lax, ops
from jax_md import space, smap, energy, minimize, quantity, simulate

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
  
sns.set_style(style='white')

# details on new things you dont yet know
from inspect import signature
import inspect, re


  


Experiment configuration

In [14]:
N = 2600
dimension = 3
box_size = quantity.box_size_at_number_density( N, 0.8, dimension)
dt = 5e-3
displacement, shift = space.periodic(box_size) 

steps = 100000
write_every = 100

max_temp = 0.005
min_temp = 0.001

# kT = lambda t: np.where(t < 5000.0 * dt, 0.001, 0.005)
# kT = lambda t: np.where(t < 5000.0 * dt, 0.001, 0.005)
# kT = lambda t: round(t * ( (max_temp - min_temp) / steps ), 3)
# kT = lambda t: np.round( t * ( (max_temp - min_temp) / steps ) , 3)
kT = lambda t: np.where(t < 5000.0 * dt, 0.001, 0.001)

In [15]:
print(kT(0))

0.001


Helper functions

In [16]:
# relative concentration
conc_n2 = 0.78084
#conc_n2 = 0.68084
conc_o2 = 0.20946
#conc_o2 = 0.15946
conc_ar = 0.009340
# conc_ar = 0.02340
conc_co2 = 0.000407
# conc_co2 = 0.0707
# conc_co2 = 0.0207
conc_h2o = 0
conc_ch4 = 0.0000018


# kinetic diameters
kd_n2 = 3.64
kd_o2 = 3.46
kd_ar = 0
kd_co2 = 3.3
kd_h2o = 2.65
kd_ch4 = 3.8

# molecular weight
mw_n2 = 28
mw_o2 = 32
mw_ar = 0
mw_co2 = 44
mw_h2o = 18
mw_ch4 = 16

# molecular diameter
md_n2 = 0
md_o2 = 0
md_ar = 0
md_co2 = 0
md_h2o = 0
md_ch4 = 0


Helper functions

In [17]:
def step_fn(i, state_and_log):
  state, log = state_and_log

  t = i * dt

  # Log information about the simulation.
  T = quantity.temperature(state.velocity)
  log['kT'] = log['kT'].at[i].set(T)
  H = simulate.nvt_nose_hoover_invariant(energy_fn, state, kT(t))
  log['H'] = log['H'].at[i].set(H)
  # Record positions every `write_every` steps.
  log['position'] = lax.cond(i % write_every == 0,
                             lambda p: \
                             p.at[i // write_every].set(state.position),
                             lambda p: p,
                             log['position'])

  # Take a simulation step.
  state = apply(state, kT=kT(t))



  return state, log

In [18]:
#import numpy as np
# import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization, UtilityFunction
import warnings
warnings.filterwarnings("ignore")
# Prepare the data.
cancer = load_breast_cancer()
X = cancer["data"]
y = cancer["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                            stratify = y,
                                        random_state = 42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)# Define the black box function to optimize.

def black_box_function(C):
    # C: SVC hyper parameter to optimize for.
    model = SVC(C = C)
    model.fit(X_train_scaled, y_train)
    y_score = model.decision_function(X_test_scaled)
    f = roc_auc_score(y_test, y_score)
    return f# Set range of C to optimize for.
# bayes_opt requires this to be a dictionary.
pbounds = {"C": [0.1, 10]}# Create a BayesianOptimization optimizer,
# and optimize the given black_box_function.
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 4)

optimizer.maximize(init_points = 5, n_iter = 10)

print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  0.9979   |  9.674    |
|  2        |  0.9975   |  5.518    |
|  3        |  0.9979   |  9.73     |
|  4        |  0.9979   |  7.177    |
|  5        |  0.9979   |  7.008    |
|  6        |  0.9914   |  0.1023   |
|  7        |  0.9981   |  8.506    |
|  8        |  0.9981   |  8.15     |
|  9        |  0.9981   |  8.327    |
|  10       |  0.9981   |  8.8      |
|  11       |  0.9981   |  8.671    |
|  12       |  0.9981   |  7.974    |
|  13       |  0.9979   |  6.273    |
|  14       |  0.9981   |  8.064    |
|  15       |  0.9981   |  8.911    |
Best result: {'C': 8.505539252885074}; f(x) = 0.9981132075471698.


In [19]:
key = random.PRNGKey(0)

In [20]:
key, split = random.split(key)
R = box_size * random.uniform(split, (N, dimension), dtype=np.float64)

# The system ought to be a 50:50 mixture of two types of particles, one
# large and one small.
# sigma = np.array([[1.0, 1.2], [1.2, 1.4]])
# sigma = np.array([[0.3, 0.3], [1.2, 1.2], [0.8, 0.8]])
sigma = np.array([[0.3, 0.3], [0.5, 0.5], [0.8, 0.8], [0.7, 0.7]])


N_2 = int(N * conc_o2)
N_3 = int(N * conc_ar)
N_4 = int(N * conc_co2)


buf = 0
buf_lst = []
for i in range(0, N):

  if buf < N_4:
    buf_lst.append(0)
  elif buf < N_3:
    buf_lst.append(1)
  elif buf < N_2:
    buf_lst.append(2)    
  else:
    buf_lst.append(3)

  buf += 1

In [21]:
species = np.array(buf_lst)


In [22]:
species.shape
species

DeviceArray([0, 1, 1, ..., 3, 3, 3], dtype=int64)

In [23]:
energy_fn = energy.soft_sphere_pair(displacement, species=species, sigma=sigma)

init, apply = simulate.nvt_nose_hoover(energy_fn, shift, dt, kT(0.))

print(type(init(key, R)))
state = init(key, R)


<class 'jax_md.simulate.NVTNoseHooverState'>


In [24]:
# establish log
log = {
    'kT': np.zeros((steps,)),
    'H': np.zeros((steps,)),
    'position': np.zeros((steps // write_every,) + R.shape) 
}

In [ ]:

state, log = lax.fori_loop(0, steps, step_fn, (state, log))

R = state.position

In [ ]:
buf_lst = []
for i in species:
   
  # carbon dioxide
  if i == 0: 
    buf_lst.append(np.array([1.5, 3.2, 0.01]))

  # argon
  elif i == 1: 
    buf_lst.append(np.array([1.0, 0.2, 0.5]))   

  # oxygen
  elif i == 2: 
    buf_lst.append(np.array([3.0, 1.2, 2.5]))   

  # nitrogen
  elif i == 3: 
    buf_lst.append(np.array([0.3, .8, 0.85 ]))        


In [ ]:
from jax_md.colab_tools import renderer

diameters = sigma[species, species]

colors = np.array(buf_lst)

renderer.render(box_size,
                {
                    'particles': renderer.Sphere(log['position'], 
                                               diameters,
                                               colors)   
                                      
                },
                resolution=(800, 800))

(for example, at 63 degrees F, CO2 molecules crash together about 7 billion times per second)

A CO2 molecule is 0.33nm diameter

The diameter of an O2 molecule is 292 picometers, and that of N2 is 300 picometers

In [ ]:
log['position'].shape

In [ ]:
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5


In [ ]:
#import statistics
#print(log['position'][1000][4])
#print(min(log['position'][1000][3]))

gen = (x for x in log['position'][1000][:18] )
gen_ = (x for x in log['position'][1000][:18])
#print(sum(list(x[0] for x in gen_)) / len(list(x[0] for x in gen)))

for i in range(0,3):
  print(mean(list(x[i] for x in (x for x in log['position'][1000][:18] ))))
  print(stddev(list(x[i] for x in (x for x in log['position'][1000][:18] ))))



#print(min(x[0] for x in gen_))
#print(max(x[0] for x in gen))

In [ ]:
#import time as tm

iii = 0
max_x = 0
min_x = 10

gg = tm.time()
for i in species:
  if i == 0:

    for ii in range(0, len(log['position'])):
      iii += 1

      min_x = min(log['position'][ii][i][1] for x in log['position'][ii][i])

      # ff = log['position'][ii][i][0]

      #if log['position'][ii][i][0] > max_x:
      #  max_x = log['position'][ii][i][0]

      #if log['position'][ii][i][0] < min_x:
      #  min_x = log['position'][ii][i][0]        

      # print(log['position'][ii][i])


print(tm.time() - gg)
print(iii)
print(max_x)
print(min_x)


In [ ]:
len(log['position'])

In [ ]:
# ## object details

# print("Let review what we've done"); print()

# objs = [energy_fn, init, apply, key, R, log]

# for obj in objs:

#   # function deets
#   try:

#     var_nm = [key for key, value in locals().items() if value == obj]
#     print(var_nm[0])

#     print('   ', obj.__name__ ,' type -', type(obj))
#     print('   ', str(signature(obj)))
#     print('')
#   except:
#     pass

#   # dict deets
#   try:
#     var_nm = [key for key, value in locals().items() if value == obj]
#     print(' type -', type(obj))
#     print('   ', str(signature(obj)))
#     print('   ', obj.shape)
#     print('')
#   except:
#     pass

#   # array deets
#   try:
#     print(' type -', type(obj))
#     print('   ', str(signature(obj)))
#     print('   ', obj.shape)
#     print('')
#   except:
#     pass





In [ ]:
# '''
# import imageio
# import jax.numpy as jnp

# def make_from_image(filename, size_in_pixels):
#   position = []
#   angle = []
#   color = []

#   img = imageio.imread(filename)

#   scale = 2**(1/6)
#   ratio = jnp.sqrt(1 - 0.25)
#   for i, y in enumerate(range(0, img.shape[0], size_in_pixels)):
#     for x in range(0, img.shape[1], size_in_pixels):
#       r, g, b, a = img[y, x]
#       if a == 255:
#         hshift = size_in_pixels * (i % 2) / 2.0
#         position += [[scale * (x + hshift) / size_in_pixels, scale * (img.shape[0] - y) / size_in_pixels * ratio]]
#         color += [[r / 255, g / 255, b / 255]]
#   img_size = jnp.array(img.shape[:2]).T / size_in_pixels * scale
#   box_size = jnp.max(img_size) * 1.5
#   position = jnp.array(position, jnp.float64) + box_size / 2.0 - img_size / 2
#   color = jnp.array(color, jnp.float64)

#   return box_size, position, color
#   '''

In [ ]:
# '''
# box, positions, colors = make_from_image('mfi_three.png', 24)
# '''

In [ ]:
# '''
# from jax_md.colab_tools import renderer

# renderer.render(box,
#                 renderer.Disk(positions, color=colors))

#                 '''

In [ ]:

# 
# from jax_md import space

# displacement_fn, shift_fn = space.periodic(box)
# 

In [ ]:


# positions[0]

In [ ]:
# displacement_fn(positions[0], positions[-1])

In [ ]:
# shift_fn(positions[0], jnp.array([10.0, 0.0]))

## Energy

"Energy" in Physics plays a similar role to "Loss" in machine learning. 

Write down an energy function between two grains of sand, $\epsilon(r)$. 

The total energy will be the sum of all pairs of energies.

$$E = \sum_{i,j} \epsilon(r_{ij})$$

where $r_{ij}$ is the distance between grain $i$ and grain $j$.


We want to model wet sand:

*   Grains are hard (no interpenetration).
*   Grains stick together a little bit.
*   Grains far away from one another don't notice each other.

In [ ]:
# from jax_md import energy

# rs = jnp.linspace(0.5, 2.5)
# plt.plot(rs, energy.lennard_jones(rs))

# plt.ylim([-1, 1])
# plt.xlim([0, 2.5])
# plt.xlabel('$r_{ij}$')
# plt.ylabel('$\\epsilon$')

In [ ]:
# sand_energy = energy.lennard_jones_pair(displacement_fn)

# sand_energy(positions)

## Simulate

In [ ]:
# from jax import random

# simulation_steps = 10000
# write_every = 50
# key = random.PRNGKey(1)

In [ ]:
# from jax_md import simulate
# from jax import jit

# init_fn, step_fn = simulate.nvt_langevin(sand_energy, shift_fn, dt=5e-3, kT=0.0, gamma=1e-2)

# sand = init_fn(key, positions)
# step_fn = jit(step_fn)

In [ ]:

# trajectory = []

# for i in range(simulation_steps):
#  if i % write_every == 0:
#    trajectory += [sand.position]
    
#  sand = step_fn(sand)

# trajectory = jnp.stack(trajectory)

In [ ]:
# renderer.render(box, renderer.Disk(trajectory, color=colors))